# Defining Resources

In the employed methodology, any stream can essentially be treated as a resource. As such, any system can be modeled using the general purpose object `Resource`. However, drawing a semantic distinction based on utility can make it easier to understand the model  

## General Resource

Resources can be declared as shown below

In [6]:
from energia import Model, Resource, Periods, si_units

m = Model(init=[si_units])

m.q = Periods(label='Quarter')
m.y = 4 * m.q
m.y.label = 'Year'

m.USD = Resource(label='US Dollar')
m.h2o = Resource(label='Water')
m.co2 = Resource(label='Carbon Dioxide')
m.cement = Resource(label='Cement')
m.barrenland = Land(label='Barren Land')

Limits (lower and upper bounds) to stream flows can be provided based on data

In [7]:
m.h2o.consume <= [200, 150, 300, 250]

--- General Resource Balance for h2o in (l, q): initializing constraint, adding consume(h2o, l, q)
    Completed in 0.00018668174743652344 seconds
--- Binding consume in domain (h2o, l, q)
    Completed in 0.0001087188720703125 seconds


This initiates both a General Resource `Balance` constraint (C[0]) and a `Bind` constraint. As the limit (upper bound) in this case was provided as a list of size 4, a constraint was written for every quarter (`m.q`). `Component.show()` can be used at any time to see the constraints the component is involved in. Use True for a descriptive print

In [8]:
m.h2o.show(True)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

`Resource` comes with all the bells and whistles but provides no stop gaps. For example, I can set a limit (lower bound in this case) on how much USD (a currency) I want to dispense annually.

In [9]:
m.USD.release >= 2000

--- General Resource Balance for USD in (l, y): initializing constraint, adding release(USD, l, y)
    Completed in 0.00010156631469726562 seconds
--- Binding release in domain (USD, l, y)
    Completed in 8.225440979003906e-05 seconds


Only one constraint is written for the year since a singular value was provided which matches size 1 of `m.y`

In [10]:
m.USD.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Using exclusive subsets can help restrict their behavior to logical possibilities. As seen below, an arrow is thrown when I try to consume INR

In [11]:
from energia import Model, Currency

m = Model()
m.INR = Currency(label='Indian Rupee')
m.INR.consume >= 20

AttributeError: 'Currency' object has no attribute 'consume'

## Resource Subsets

There are many different types of resources, each with their limited set of `Aspect`. Refer to the documentation to see them. The distinction between `Resource` and `Material` is especially tenuous 

In [12]:
from energia import (
    Model,
    Resource,
    Currency,
    Material,
    Land,
    Emission,
    Periods,
    si_units,
)

m = Model(init=[si_units])

m.q = Periods(label='Quarter')
m.y = 4 * m.q
m.y.label = 'Year'

m.USD = Currency(label='US Dollar')
m.h2o = Resource(label='Water')
m.co2 = Resource(label='Carbon Dioxide')
m.cement = Material(label='Cement')
m.barrenland = Land(label='Barren Land')
m.farmland = Land(label='Farm Land')

## Resource Conversion

Any operation involving resource (irrespective of subset) forms a `Conversion` object

In [13]:
conv = m.h2o - 3 * m.co2 + m.farmland / 4
conv

η(None)

The conversion balance can be viewed using `Conversion.conversion`

In [14]:
conv.conversion

{h2o: 1, co2: -3, farmland: 0.25}

Note that `Process`es do not check the type of `Resource` object provided. This lends considerable modeling flexibility; operations can be defined to convert resources in materials, land into other type with resources needed to do the same, etc. While this opens the door for numerous possibilities, the user needs to be careful in assuring that the conversion makes sense. 